In [38]:
#Предобработка данных

In [39]:
import pandas as pd

In [40]:
data = pd.read_csv('yandex_music_project.csv')

In [56]:
data = data.rename(columns={'  userID': 'user_id', 'Track': 'track', 'Artist': 'artist', 'Genre': 'genre', '  City  ': 'city', 'Time':'time','Day':'day'})

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  65079 non-null  object
 1   track    63848 non-null  object
 2   artist   57876 non-null  object
 3   genre    63881 non-null  object
 4   city     65079 non-null  object
 5   time     65079 non-null  object
 6   day      65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


In [58]:
data.head()

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [62]:
print(data.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


In [60]:
colums_to_replace = ['track', 'artist','genre']

In [61]:
for col in colums_to_replace:
    data[col] = data[col].fillna('unknown')

In [68]:
duplicated_data = data[data.duplicated()].head()

In [65]:
data = data.drop_duplicates()

In [66]:
data = data.reset_index()

In [84]:
data['genre'] = data['genre'].replace('электроника', 'electronic')
data['genre'] = data['genre'].replace('hip', 'hiphop')
data['genre'] = data['genre'].replace('hop', 'hiphop')

In [89]:
wrong_genres = ['hip','hop','hip-hop']
correct_genre = 'hiphop'
def replace_wrong_genre(wrong_genres, correct_genre):
    for genre in wrong_genres:
        data['genre'] = data['genre'].replace(genre, coorect_genre)

In [96]:
genre_unique = list(data['genre'].unique())

In [107]:
sorted_genre = data.sort_values(by='genre')

In [105]:
data = data.reset_index(drop=True)

In [120]:
group_listeniers_by_time = data.groupby('city')['time'].count()

In [136]:
group_listeniers_by_day = data.groupby('day')['city'].count()

In [137]:
group_listeniers_by_day

day
Friday       21840
Monday       21354
Wednesday    18059
Name: city, dtype: int64

In [155]:
def number_tracks(day, city):
    track_list = data[data['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    print(track_list_count)
    return track_list_count

In [156]:
monday_moscow = number_tracks('Monday', 'Moscow')
wednesday_moscow = number_tracks('Wednesday', 'Moscow')
friday_moscow = number_tracks('Friday', 'Moscow')

monday_saint_petersburg = number_tracks('Monday', 'Saint-Petersburg')
wednesday_saint_petersburg = number_tracks('Wednesday', 'Saint-Petersburg')
friday_saint_petersburg = number_tracks('Friday', 'Saint-Petersburg')


15740
11056
15945
5614
7003
5895


In [162]:
atlas = [['Moscow',monday_moscow, wednesday_moscow, friday_moscow],
        ['Saint-Petersburg',monday_saint_petersburg, wednesday_saint_petersburg,friday_saint_petersburg]]

In [163]:
data_about_cities = pd.DataFrame(data = atlas, columns=['city', 'monday', 'wednesday','friday'])

In [262]:
data_about_cities

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


In [263]:
### Музыка в начале и в конце недели

In [255]:
moscow_general = data[data['city'] == 'Moscow']

In [264]:
spb_general = data[data['city'] == 'Saint-Petersburg']

In [257]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['track'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted

In [283]:
moscow_top_genre_monday_early = genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
saint_top_genre_monday_early = genre_weekday(spb_general, 'Monday', '07:00', '11:00')

moscow_top_genre_friday_after = genre_weekday(moscow_general, 'Friday', '17:00', '23:00')
saint_top_genre_friday_after = genre_weekday(spb_general, 'Friday', '17:00', '23:00')

In [287]:
moscow_top_genre_friday_after.head(10)

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: track, dtype: int64

In [286]:
saint_top_genre_friday_after.head(10)

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: track, dtype: int64

In [296]:
grouped_moscow_general = moscow_general.groupby('genre')['track'].count()
grouped_saint_general = spb_general.groupby('genre')['track'].count()

In [297]:
grouped_moscow_general = grouped_moscow_general.sort_values(ascending=False)
grouped_saint_general = grouped_saint_general.sort_values(ascending=False)

In [300]:
grouped_saint_general.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: track, dtype: int64

In [301]:
grouped_moscow_general.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2095
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: track, dtype: int64